# line_profilerの使い方

In [2]:
import line_profiler
import numpy as np


def func():
    for i in range(5):
        a = np.empty([i*100, 1000])


def test():
    func()


profile = line_profiler.LineProfiler()
profile.add_function(func)
profile.runcall(test)
profile.print_stats(output_unit=1e-9)


Timer unit: 1e-09 s

Total time: 6.39e-05 s
File: C:\Users\fujikawa\AppData\Local\Temp\ipykernel_5840\671070373.py
Function: func at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def func():
     6         5       3400.0    680.0      5.3      for i in range(5):
     7         5      60500.0  12100.0     94.7          a = np.empty([i*100, 1000])



### ループに使用

In [1]:
import line_profiler
import numpy as np


def func():
    for i in range(5):
        a = np.empty([i*100, 1000])


def test():
    loop_count = 1000
    for i in range(loop_count):
        func()



profile = line_profiler.LineProfiler()

# profile.enable_by_count()
profile.add_function(func)
profile.add_function(test)
# profile.disable_by_count()

profile.runcall(test)
profile.print_stats(output_unit=1e-9)



Timer unit: 1e-09 s

Total time: 0.0208671 s
File: C:\Users\fujikawa\AppData\Local\Temp\ipykernel_5840\2983318294.py
Function: func at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def func():
     6      5000    1292900.0    258.6      6.2      for i in range(5):
     7      5000   19574200.0   3914.8     93.8          a = np.empty([i*100, 1000])

Total time: 0.0281482 s
File: C:\Users\fujikawa\AppData\Local\Temp\ipykernel_5840\2983318294.py
Function: test at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def test():
    11         1        400.0    400.0      0.0      loop_count = 1000
    12      1000     249200.0    249.2      0.9      for i in range(loop_count):
    13      1000   27898600.0  27898.6     99.1          func()



### スレッドに適用

In [2]:
import random
import threading
import time

from line_profiler import LineProfiler


def do_other_stuff(numbers):
    time.sleep(1)
    s = sum(numbers)


def do_stuff(numbers):
    my_thread = threading.Thread(target=do_other_stuff, args=[numbers])
    my_thread.start()
    my_thread.join()


thread_to_lp = {}


def wrapper_in_thread(func):
    def inner(*args, **kwargs):
        lp = thread_to_lp.setdefault(
            threading.current_thread(), LineProfiler())
        lp.add_function(func)
        lp.enable_by_count()
        try:
            return func(*args, **kwargs)
        finally:
            lp.disable_by_count()

    return inner


do_other_stuff = wrapper_in_thread(do_other_stuff)
do_stuff = wrapper_in_thread(do_stuff)

numbers = [random.randint(1, 100) for i in range(1000)]
do_stuff(numbers)

for lp in thread_to_lp.values():
    lp.print_stats()


Timer unit: 1e-07 s

Total time: 1.00096 s
File: C:\Users\fujikawa\AppData\Local\Temp\ipykernel_14508\4080591567.py
Function: do_stuff at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def do_stuff(numbers):
    14         1        568.0    568.0      0.0      my_thread = threading.Thread(target=do_other_stuff, args=[numbers])
    15         1       3936.0   3936.0      0.0      my_thread.start()
    16         1   10005097.0 10005097.0    100.0      my_thread.join()

Timer unit: 1e-07 s

Total time: 1.00053 s
File: C:\Users\fujikawa\AppData\Local\Temp\ipykernel_14508\4080591567.py
Function: do_other_stuff at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def do_other_stuff(numbers):
     9         1   10005182.0 10005182.0    100.0      time.sleep(1)
    10         1        111.0    111.0      0.0      s = sum(numbers)

